In [1]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

from dash.dependencies import Input, Output

In [2]:
df = pd.read_csv("2020_LoL_esports_match_data_from_OraclesElixir_20210730.csv")

/home/tuedo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.shape

(115140, 117)

In [4]:
print(df.columns.to_list())

['gameid', 'datacompleteness', 'url', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'playerid', 'side', 'position', 'player', 'team', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes', 'infernals', 'mountains', 'clouds', 'oceans', 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'firstbaron', 'barons', 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'inhibitors', 'opp_inhibitors', 'damagetochampions', 'dpm', 'damageshare', 'damagetakenperminute', 'damagemitigatedperminute', 'wardsplaced', 'wpm', 'wardskilled', 'wcpm', 'controlwardsbought', 'visionscore', 'vspm', 'totalgold',

In [5]:
lcs_df = df[(df["league"] == "LCS")];
lcs_df.shape

(3168, 117)

In [6]:
lcs_pick_dict = dict()

for champ in lcs_df['champion']:
    lcs_pick_dict[champ] = lcs_pick_dict.get(champ,0) + 1
    
lcs_ban_dict = dict()

for ban in lcs_df[['ban1','ban2','ban3','ban4','ban5']]:
    for champ in lcs_df[ban]:
        lcs_ban_dict[champ] = lcs_ban_dict.get(champ,0) + 1

for key in lcs_ban_dict:
    lcs_ban_dict[key] = int(lcs_ban_dict[key] / 6)

lcs_pick_df = pd.DataFrame.from_dict({'champion' : list(lcs_pick_dict.keys()), 'number of picks' : list(lcs_pick_dict.values())})
lcs_pick_df = lcs_pick_df.dropna()
lcs_pick_df = lcs_pick_df.sort_values('number of picks', ascending = False)

lcs_ban_df = pd.DataFrame.from_dict({'champion' : list(lcs_ban_dict.keys()), 'number of bans' : list(lcs_ban_dict.values())})
lcs_ban_df = lcs_ban_df.dropna()
lcs_ban_df = lcs_ban_df.sort_values('number of bans', ascending = False)

lcs_pick_ban_df = pd.merge(lcs_pick_df, lcs_ban_df, how="outer", on=["champion"]).fillna(0)
lcs_pick_ban_df["pick and ban"] = lcs_pick_ban_df.loc[:,["number of picks", "number of bans"]].sum(axis=1)

# There were 264 games played in LCS 2020 (counted outside of this notebook)
lcs_pick_ban_df["P%"] = 100 * lcs_pick_ban_df["number of picks"] / 264
lcs_pick_ban_df["B%"] = 100 * lcs_pick_ban_df["number of bans"] / 264
lcs_pick_ban_df = lcs_pick_ban_df.sort_values("pick and ban", ascending=False)

In [7]:
lec_df = df[(df["league"] == "LEC")];
lec_df.shape

(2892, 117)

In [8]:
lec_pick_dict = dict()

for champ in lec_df['champion']:
    lec_pick_dict[champ] = lec_pick_dict.get(champ,0) + 1
    
lec_ban_dict = dict()

for ban in lec_df[['ban1','ban2','ban3','ban4','ban5']]:
    for champ in lec_df[ban]:
        lec_ban_dict[champ] = lec_ban_dict.get(champ,0) + 1

for key in lec_ban_dict:
    lec_ban_dict[key] = int(lec_ban_dict[key] / 6)

lec_pick_df = pd.DataFrame.from_dict({'champion' : list(lec_pick_dict.keys()), 'number of picks' : list(lec_pick_dict.values())})
lec_pick_df = lec_pick_df.dropna()
lec_pick_df = lec_pick_df.sort_values('number of picks', ascending = False)

lec_ban_df = pd.DataFrame.from_dict({'champion' : list(lec_ban_dict.keys()), 'number of bans' : list(lec_ban_dict.values())})
lec_ban_df = lec_ban_df.sort_values('number of bans', ascending = False)

lec_pick_ban_df = pd.merge(lec_pick_df, lec_ban_df, how="outer", on=["champion"]).fillna(0)
lec_pick_ban_df["pick and ban"] = lec_pick_ban_df.loc[:,["number of picks", "number of bans"]].sum(axis=1)

# There were 241 games played in LEC 2020 (counted outside of this notebook)
lec_pick_ban_df["P%"] = 100 * lec_pick_ban_df["number of picks"] / 241
lec_pick_ban_df["B%"] = 100 * lec_pick_ban_df["number of bans"] / 241

lec_pick_ban_df = lec_pick_ban_df.sort_values("pick and ban", ascending=False)

In [9]:
lec_pick_ban_df = lec_pick_ban_df.drop(labels=120, axis=0) # manually removing buggy entry

In [10]:
lck_df = df[(df["league"] == "LCK")];
lck_df.shape

(5676, 117)

In [11]:
lck_pick_dict = dict()

for champ in lck_df['champion']:
    lck_pick_dict[champ] = lck_pick_dict.get(champ,0) + 1
    
lck_ban_dict = dict()

for ban in lck_df[['ban1','ban2','ban3','ban4','ban5']]:
    for champ in lck_df[ban]:
        lck_ban_dict[champ] = lck_ban_dict.get(champ,0) + 1

for key in lck_ban_dict:
    lck_ban_dict[key] = int(lck_ban_dict[key] / 6)

lck_pick_df = pd.DataFrame.from_dict({'champion' : list(lck_pick_dict.keys()), 'number of picks' : list(lck_pick_dict.values())})
lck_pick_df = lck_pick_df.dropna()
lck_pick_df = lck_pick_df.sort_values('number of picks', ascending = False)

lck_ban_df = pd.DataFrame.from_dict({'champion' : list(lck_ban_dict.keys()), 'number of bans' : list(lck_ban_dict.values())})
lck_ban_df = lck_ban_df.dropna()
lck_ban_df = lck_ban_df.sort_values('number of bans', ascending = False)

lck_pick_ban_df = pd.merge(lck_pick_df, lck_ban_df, how="outer", on=["champion"]).fillna(0)
lck_pick_ban_df["pick and ban"] = lck_pick_ban_df.loc[:,["number of picks", "number of bans"]].sum(axis=1)

# There were 473 games played in LCK 2020 (counted outside of this notebook)
lck_pick_ban_df["P%"] = 100 * lck_pick_ban_df["number of picks"] / 473
lck_pick_ban_df["B%"] = 100 * lck_pick_ban_df["number of bans"] / 473
lck_pick_ban_df = lck_pick_ban_df.sort_values("pick and ban", ascending=False)

In [12]:
lpl_df = df[(df["league"] == "LPL")];
lpl_df.shape

(8712, 117)

In [13]:
lpl_pick_dict = dict()

for champ in lpl_df['champion']:
    lpl_pick_dict[champ] = lpl_pick_dict.get(champ,0) + 1
    
lpl_ban_dict = dict()

for ban in lpl_df[['ban1','ban2','ban3','ban4','ban5']]:
    for champ in lpl_df[ban]:
        lpl_ban_dict[champ] = lpl_ban_dict.get(champ,0) + 1

for key in lpl_ban_dict:
    lpl_ban_dict[key] = int(lpl_ban_dict[key] / 6)

lpl_pick_df = pd.DataFrame.from_dict({'champion' : list(lpl_pick_dict.keys()), 'number of picks' : list(lpl_pick_dict.values())})
lpl_pick_df = lpl_pick_df.dropna()
lpl_pick_df = lpl_pick_df.sort_values('number of picks', ascending = False)

lpl_ban_df = pd.DataFrame.from_dict({'champion' : list(lpl_ban_dict.keys()), 'number of bans' : list(lpl_ban_dict.values())})
lpl_ban_df = lpl_ban_df.dropna()
lpl_ban_df = lpl_ban_df.sort_values('number of bans', ascending = False)

lpl_pick_ban_df = pd.merge(lpl_pick_df, lpl_ban_df, how="outer", on=["champion"]).fillna(0)
lpl_pick_ban_df["pick and ban"] = lpl_pick_ban_df.loc[:,["number of picks", "number of bans"]].sum(axis=1)

# There were 726 games played in LPL 2020 (counted outside of this notebook)
lpl_pick_ban_df["P%"] = 100 * lpl_pick_ban_df["number of picks"] / 726
lpl_pick_ban_df["B%"] = 100 * lpl_pick_ban_df["number of bans"] / 726
lpl_pick_ban_df = lpl_pick_ban_df.sort_values("pick and ban", ascending=False)

In [14]:
# Setting up figures

lcs_picks = px.bar(lcs_pick_df, x="champion", y="number of picks");
lcs_bans = px.bar(lcs_ban_df, x="champion", y="number of bans");

lcs_pick_ban_percentage = px.bar(lcs_pick_ban_df, x="champion", y=["P%","B%"]);
lcs_pick_ban_percentage.update_layout(yaxis_title="P/B %", xaxis=dict(tickmode='linear'));
lcs_pick_ban_percentage.update_yaxes(range=[0,100]);


lec_picks = px.bar(lec_pick_df, x="champion", y="number of picks");
lec_bans = px.bar(lec_ban_df, x="champion", y="number of bans");

lec_pick_ban_percentage = px.bar(lec_pick_ban_df, x="champion", y=["P%","B%"]);
lec_pick_ban_percentage.update_layout(yaxis_title="P/B %", xaxis=dict(tickmode='linear'));
lec_pick_ban_percentage.update_yaxes(range=[0,100]);

lck_picks = px.bar(lck_pick_df, x="champion", y="number of picks");
lck_bans = px.bar(lck_ban_df, x="champion", y="number of bans");

lck_pick_ban_percentage = px.bar(lck_pick_ban_df, x="champion", y=["P%","B%"]);
lck_pick_ban_percentage.update_layout(yaxis_title="P/B %", xaxis=dict(tickmode='linear'));
lck_pick_ban_percentage.update_yaxes(range=[0,100]);

lpl_picks = px.bar(lpl_pick_df, x="champion", y="number of picks");
lpl_bans = px.bar(lpl_ban_df, x="champion", y="number of bans");

lpl_pick_ban_percentage = px.bar(lpl_pick_ban_df, x="champion", y=["P%","B%"]);
lpl_pick_ban_percentage.update_layout(yaxis_title="P/B %", xaxis=dict(tickmode='linear'));
lpl_pick_ban_percentage.update_yaxes(range=[0,100]);

In [15]:
# Each region, we will show:
# 1. combined kills per minute (cpkm and dpm_
# 2. vision (wpm and vspm)
# 3. 10 minute leads (gold diff at 10, xp diff at 10)
# 4. 15 minute leads (gold diff at 15, xp diff at 15)

In [16]:
region_compare_df = pd.DataFrame({'region' : ['LCS', 'LEC', 'LCK', 'LPL']});

In [17]:
region_compare_df['combined kills per minute per game'] = [lcs_df['ckpm'].sum() / 12 / 264, 
    lec_df['ckpm'].sum() / 12 / 241, 
    lck_df['ckpm'].sum() / 12 / 473, 
    lpl_df['ckpm'].sum() / 12 / 726];

region_compare_df['combined damage per minute per game'] = [lcs_df['dpm'].sum() / 2 / 264, 
    lec_df['dpm'].sum() / 2 / 241,
    lck_df['dpm'].sum() / 2 / 473,
    lpl_df['dpm'].sum() / 2 / 726];

region_compare_df['combined wards per minute per game'] = [lcs_df['wpm'].sum() / 2 / 264, 
    lec_df['wpm'].sum() / 2 / 241,
    lck_df['wpm'].sum() / 2 / 473,
    lpl_df['wpm'].sum() / 2 / 726];

region_compare_df['combined vision score per minute per game'] = [lcs_df['vspm'].sum() / 2 / 264,
    lec_df['vspm'].sum() / 2 / 241,
    lck_df['vspm'].sum() / 2 / 473, 
    lpl_df['vspm'].sum() / 2 / 726];

# Every game has blue team as playerid 100 and red team as playerid 200

region_compare_df['average magnitude of gold diff at 10'] = [lcs_df[lcs_df['playerid'] == 100]['golddiffat10'].abs().sum() / 264,
    lec_df[lec_df['playerid'] == 100]['golddiffat10'].abs().sum() / 241,
    lck_df[lck_df['playerid'] == 100]['golddiffat10'].abs().sum() / 473,
    lpl_df[lpl_df['playerid'] == 100]['golddiffat10'].abs().sum() / 726];

region_compare_df['average magnitude of xp diff at 10'] = [lcs_df[lcs_df['playerid'] == 100]['xpdiffat10'].abs().sum() / 264,
    lec_df[lec_df['playerid'] == 100]['xpdiffat10'].abs().sum() / 241,
    lck_df[lck_df['playerid'] == 100]['xpdiffat10'].abs().sum() / 473,
    lpl_df[lpl_df['playerid'] == 100]['xpdiffat10'].abs().sum() / 726];

region_compare_df['average magnitude of gold diff at 15'] = [lcs_df[lcs_df['playerid'] == 100]['golddiffat15'].abs().sum() / 264,
    lec_df[lec_df['playerid'] == 100]['golddiffat15'].abs().sum() / 241,
    lck_df[lck_df['playerid'] == 100]['golddiffat15'].abs().sum() / 473,
    lpl_df[lpl_df['playerid'] == 100]['golddiffat15'].abs().sum() / 726];

region_compare_df['average magnitude of xp diff at 15'] = [lcs_df[lcs_df['playerid'] == 100]['xpdiffat15'].abs().sum() / 264,
    lec_df[lec_df['playerid'] == 100]['xpdiffat15'].abs().sum() / 241,
    lck_df[lck_df['playerid'] == 100]['xpdiffat15'].abs().sum() / 473,
    lpl_df[lpl_df['playerid'] == 100]['xpdiffat15'].abs().sum() / 726];

In [30]:
region_compare_df.head()

,region,combined kills per minute per game,combined damage per minute per game,combined wards per minute per game,combined vision score per minute per game,average magnitude of gold diff at 10,average magnitude of xp diff at 10,average magnitude of gold diff at 15,average magnitude of xp diff at 15
0,LCS,0.670545,3634.653071,6.567769,14.729978,977.643939,813.227273,2042.068182,1421.371212
1,LEC,0.772378,3693.347330,6.785695,14.779258,960.157676,788.124481,1938.680498,1512.120332
2,LCK,0.711884,3620.683951,7.392696,15.850278,1060.701903,840.086681,2233.012685,1605.678647
3,LPL,0.822672,3780.444494,6.719547,13.191931,1153.479339,897.406336,2150.699725,1529.290634


In [19]:
# Setting up figures
ckpm = px.bar(region_compare_df, x='region', y='combined kills per minute per game', color='region');
cdpm = px.bar(region_compare_df, x='region', y='combined damage per minute per game', color='region');

cwpm = px.bar(region_compare_df, x='region', y='combined wards per minute per game', color='region');
cvspm = px.bar(region_compare_df, x='region', y='combined vision score per minute per game', color='region');

gd10 = px.bar(region_compare_df, x='region', y='average magnitude of gold diff at 10', color='region');
gd15 = px.bar(region_compare_df, x='region', y='average magnitude of gold diff at 15', color='region');

xpd10 = px.bar(region_compare_df, x='region', y='average magnitude of xp diff at 10', color='region');
xpd15 = px.bar(region_compare_df, x='region', y='average magnitude of xp diff at 15', color='region');

In [31]:
# The data is organized by role very well
top_df = pd.concat([lcs_df[(lcs_df["playerid"] == 1) | (lcs_df["playerid"] == 6)],
                   lec_df[(lec_df["playerid"] == 1) | (lec_df["playerid"] == 6)],
                   lck_df[(lck_df["playerid"] == 1) | (lck_df["playerid"] == 6)],
                   lpl_df[(lpl_df["playerid"] == 1) | (lpl_df["playerid"] == 6)]])

jg_df = pd.concat([lcs_df[(lcs_df["playerid"] == 2) | (lcs_df["playerid"] == 7)],
                   lec_df[(lec_df["playerid"] == 2) | (lec_df["playerid"] == 7)],
                   lck_df[(lck_df["playerid"] == 2) | (lck_df["playerid"] == 7)],
                   lpl_df[(lpl_df["playerid"] == 2) | (lpl_df["playerid"] == 7)]])

mid_df = pd.concat([lcs_df[(lcs_df["playerid"] == 3) | (lcs_df["playerid"] == 6)],
                   lec_df[(lec_df["playerid"] == 3) | (lec_df["playerid"] == 6)],
                   lck_df[(lck_df["playerid"] == 3) | (lck_df["playerid"] == 6)],
                   lpl_df[(lpl_df["playerid"] == 3) | (lpl_df["playerid"] == 6)]])

adc_df = pd.concat([lcs_df[(lcs_df["playerid"] == 4) | (lcs_df["playerid"] == 8)],
                   lec_df[(lec_df["playerid"] == 4) | (lec_df["playerid"] == 8)],
                   lck_df[(lck_df["playerid"] == 4) | (lck_df["playerid"] == 8)],
                   lpl_df[(lpl_df["playerid"] == 4) | (lpl_df["playerid"] == 8)]])

sup_df = pd.concat([lcs_df[(lcs_df["playerid"] == 5) | (lcs_df["playerid"] == 10)],
                   lec_df[(lec_df["playerid"] == 5) | (lec_df["playerid"] == 10)],
                   lck_df[(lck_df["playerid"] == 5) | (lck_df["playerid"] == 10)],
                   lpl_df[(lpl_df["playerid"] == 5) | (lpl_df["playerid"] == 10)]])

In [42]:
role_compare_df = pd.DataFrame({'role' : ['top','jg','mid','adc','sup']})

In [49]:
role_compare_df['first blood %'] = [top_df['firstblood'].mean(), jg_df['firstblood'].mean(),
                                      mid_df['firstblood'].mean(), adc_df['firstblood'].mean(),
                                      sup_df['firstblood'].mean()];

role_compare_df['first blood error'] = [top_df['firstblood'].std(), jg_df['firstblood'].std(),
                                      mid_df['firstblood'].std(), adc_df['firstblood'].std(),
                                      sup_df['firstblood'].std()];

role_compare_df['damage share'] = [top_df['damageshare'].mean(), jg_df['damageshare'].mean(),
                                      mid_df['damageshare'].mean(), adc_df['damageshare'].mean(),
                                      sup_df['damageshare'].mean()];

role_compare_df['damage share error'] = [top_df['damageshare'].std(), jg_df['damageshare'].std(),
                                      mid_df['damageshare'].std(), adc_df['damageshare'].std(),
                                      sup_df['damageshare'].std()];

role_compare_df['earned gold share'] = [top_df['earnedgoldshare'].mean(), jg_df['earnedgoldshare'].mean(),
                                      mid_df['earnedgoldshare'].mean(), adc_df['earnedgoldshare'].mean(),
                                      sup_df['earnedgoldshare'].mean()];

role_compare_df['earned gold share error'] = [top_df['earnedgoldshare'].std(), jg_df['earnedgoldshare'].std(),
                                      mid_df['earnedgoldshare'].std(), adc_df['earnedgoldshare'].std(),
                                      sup_df['earnedgoldshare'].std()];

top_df['dpg'] = top_df['damagetochampions'] / top_df['earnedgold'];
jg_df['dpg'] = jg_df['damagetochampions'] / jg_df['earnedgold'];
mid_df['dpg'] = mid_df['damagetochampions'] / mid_df['earnedgold'];
adc_df['dpg'] = adc_df['damagetochampions'] / adc_df['earnedgold'];
sup_df['dpg'] = sup_df['damagetochampions'] / sup_df['earnedgold'];

role_compare_df['damage per gold'] = [top_df['dpg'].mean(), jg_df['dpg'].mean(),
                                      mid_df['dpg'].mean(), adc_df['dpg'].mean(),
                                      sup_df['dpg'].mean()];

role_compare_df['damage per gold error'] = [top_df['dpg'].std(), jg_df['dpg'].std(),
                                      mid_df['dpg'].std(), adc_df['dpg'].std(),
                                      sup_df['dpg'].std()];

In [50]:
role_compare_df.head()

,role,first blood %,first blood error,damage share,damage share error,earned gold share,earned gold share error,damage per gold,damage per gold error
0,top,0.224584,0.417372,0.228231,0.068409,0.223235,0.036217,1.732315,0.573059
1,jg,0.390327,0.487899,0.140553,0.056837,0.176198,0.030322,1.328235,0.501532
2,mid,0.225816,0.418183,0.247410,0.073366,0.230676,0.036050,1.817698,0.608759
3,adc,0.207332,0.405458,0.275833,0.076593,0.255259,0.041440,1.846522,0.621541
4,sup,0.239680,0.426954,0.080886,0.037352,0.092193,0.024457,1.515509,0.759598


In [51]:
# Setting up figures
first_blood = px.scatter(role_compare_df, x='role', 
                         y='first blood %', color='role', error_y='first blood error', 
                         error_y_minus='first blood error')

damage_share = px.scatter(role_compare_df, x='role', 
                         y='damage share', color='role', error_y='damage share error', 
                         error_y_minus='damage share error')

earned_gold_share = px.scatter(role_compare_df, x='role', 
                         y='earned gold share', color='role', error_y='earned gold share error', 
                         error_y_minus='earned gold share error')

damage_per_gold = px.scatter(role_compare_df, x='role', 
                         y='damage per gold', color='role', error_y='damage per gold error', 
                         error_y_minus='damage per gold error')

In [52]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Lol Esports 2020 Visualizer'),
    
    html.H3('Pick-Ban by Region'),
    
    dcc.Dropdown(id='region-dropdown', options=[
        {'label' : 'LCS', 'value' : 'lcs'},
        {'label' : 'LEC', 'value' : 'lec'},
        {'label' : 'LCK', 'value' : 'lck'},
        {'label' : 'LPL', 'value' : 'lpl'},
    ], value='lcs',),
    
    html.Div(id='region-num-games'),
       
    dcc.Graph(
        id='pick-graph',
        figure=lcs_picks
    ),
    
        
    dcc.Graph(
        id='ban-graph',
        figure=lcs_bans
    ),
    
        
    dcc.Graph(
        id='p-b-percentage',
        figure=lcs_pick_ban_percentage
    ),
    
    html.H3('Miscellaneous Region Statistics'),
    
    dcc.Dropdown(id='stats-dropdown', options=[
        {'label' : 'Fighting', 'value' : 'fight'},
        {'label' : 'Vision', 'value' : 'vision'},
        {'label' : 'Early Game Gold', 'value' : 'gold'},
        {'label' : 'Early Game XP', 'value' : 'xp'},
    ], value='fight',),
    
    html.Div([
        html.Div([
            dcc.Graph(id='left-graph',figure=ckpm),
        ], className="six columns"),
        
        html.Div([
            dcc.Graph(id='right-graph',figure=cdpm),
        ], className="six columns"),
    
    ], className="row"),
    
    html.H3('Miscellaneous Role Statistics'),
    
    dcc.Dropdown(id='role-dropdown', options=[
        {'label' : 'First Blood %', 'value' : 'firstblood'},
        {'label' : 'Damage Share', 'value' : 'damageshare'},
        {'label' : 'Earned Gold Share', 'value' : 'goldshare'},
        {'label' : 'Damage per Gold', 'value' : 'dpg'}
    ], value='firstblood'),
    
    dcc.Graph(
        id='role-graph',
        figure=first_blood
    ),
    
    html.Div("Match data taken from Oracle's Elixir")
])

@app.callback(
    Output('region-num-games', 'children'),
    [Input('region-dropdown', 'value')]
)
def update_num_games(value):
    if (value == 'lcs'):
        return 'There were 264 games played in LCS 2020.'
    elif (value == 'lec'):
        return 'There were 241 games played in LEC 2020.'
    elif (value == 'lck'):
        return 'There were 473 games played in LCK 2020.'
    elif (value == 'lpl'):
        return 'There were 726 games played in LPL 2020.'
    
@app.callback(
    Output('pick-graph', 'figure'),
    [Input(component_id='region-dropdown', component_property='value')]
)
def update_picks(value):
    if (value == 'lcs'):
        return lcs_picks
    elif (value == 'lec'):
        return lec_picks
    elif (value == 'lck'):
        return lck_picks
    elif (value == 'lpl'):
        return lpl_picks
@app.callback(
    Output('ban-graph', 'figure'),
    [Input(component_id='region-dropdown', component_property='value')]
)
def update_bans(value):
    if (value == 'lcs'):
        return lcs_bans
    elif (value == 'lec'):
        return lec_bans
    elif (value == 'lck'):
        return lck_bans
    elif (value == 'lpl'):
        return lpl_bans
@app.callback(
    Output('p-b-percentage', 'figure'),
    [Input(component_id='region-dropdown', component_property='value')]
)
def update_p_b_per(value):
    if (value == 'lcs'):
        return lcs_pick_ban_percentage
    elif (value == 'lec'):
        return lec_pick_ban_percentage
    elif (value == 'lck'):
        return lck_pick_ban_percentage
    elif (value == 'lpl'):
        return lpl_pick_ban_percentage

@app.callback(
    Output('left-graph', 'figure'),
    [Input('stats-dropdown', 'value')]
)
def update_left_graph(value):
    if (value == 'fight'):
        return ckpm
    elif (value == 'vision'):
        return cwpm
    elif (value == 'gold'):
        return gd10
    elif (value == 'xp'):
        return xpd10
    
@app.callback(
    Output('right-graph', 'figure'),
    [Input('stats-dropdown', 'value')]
)
def update_right_graph(value):
    if (value == 'fight'):
        return cdpm
    elif (value == 'vision'):
        return cvspm
    elif (value == 'gold'):
        return gd15
    elif (value == 'xp'):
        return xpd15
    
@app.callback(
    Output('role-graph','figure'),
    [Input('role-dropdown','value')]
)
def update_role_graph(value):
    if (value == 'firstblood'):
        return first_blood
    elif (value == 'damageshare'):
        return damage_share
    elif (value == 'goldshare'):
        return earned_gold_share
    elif (value == 'dpg'):
        return damage_per_gold
    
app.run_server(host="localhost",port=8050) # use whatever port is not 'in use', I'm not sure how this part works.

Dash app running on http://localhost:8050/
